In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from sklearn import *
import nltk,datetime

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cats=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')

test.describe()

In [ ]:
#TarihSaati hazırla

train['date']=pd.to_datetime(train['date'],format='%d.%m.%Y')
train['month']=train['date'].dt.month
train['year']=train['date'].dt.year
train=train.drop(['date','item_price'],axis=1)
train=train.groupby([c for c in train.columns if c not in ['item_cnt_day']],as_index=False)[['item_cnt_day']].sum()
train=train.rename(columns={'item_cnt_day':'item_cnt_month'})
train.head()

In [ ]:
#Aylık ortalamayı bul

shop_item_mean=train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'],as_index=False)[['item_cnt_month']].mean()
shop_item_mean=shop_item_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})

#Ortalama özelliğini train setine eklememiz yeterli

train=pd.merge(train,shop_item_mean, how='left',on=['shop_id','item_id'])
train.head()

In [ ]:
#Geçen ayı ekle

shop_prev_month=train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_prev_month=shop_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_prev_month.head()

In [ ]:
#veri kümesini eğitmek için önceki ay özelliğini ekle

train=pd.merge(train,shop_prev_month,how='left',on=['shop_id','item_id']).fillna(0.)

#Tüm öğe özelliklerini ekle

train=pd.merge(train,items,how='left',on='item_id')

#Kategori özelliklerini ekle

train=pd.merge(train,item_cats,how='left',on='item_category_id')

#mağaza özelliklerini ekle

train=pd.merge(train,shops,how='left',on='shop_id')
train.head()

In [ ]:
# Test veri seti

# Ortalama ekle

test=pd.merge(test,shop_item_mean,how='left',on=['shop_id','item_id']).fillna(0.)

#Geçen ayı ekle

test=pd.merge(test,shop_prev_month,how='left',on=['shop_id','item_id']).fillna(0.)

#Tüm öğe özelliklerini ekle

test=pd.merge(test,items,how='left',on='item_id')

#Kategori özelliklerini ekle

test=pd.merge(test,item_cats,how='left',on='item_category_id')

#mağaza özelliklerini ekle

test=pd.merge(test,shops,how='left',on='shop_id')
test['item_cnt_month']=0.
test.head()

In [ ]:
#etiket kodlaması

for c in ['shop_name','item_name','item_category_name']:
    lbl=preprocessing.LabelEncoder()
    lbl.fit(list(train[c].unique())+list(test[c].unique()))
    train[c]=lbl.transform(train[c].astype(str))
    test[c]=lbl.transform(test[c].astype(str))
    print(c)

In [ ]:
#Random forest algorithm 

col = [c for c in train.columns if c not in ['item_cnt_month']]
x1=train[train['date_block_num']<33]
y1=np.log1p(x1['item_cnt_month'].clip(0.,20.))
x1=x1[col]
x2=train[train['date_block_num']==33]
y2=np.log1p(x2['item_cnt_month'].clip(0.,20.))
x2=x2[col]

reg=ensemble.ExtraTreesRegressor(n_estimators=40,n_jobs=-1,max_depth=15,random_state=18)
reg.fit(x1,y1)
print('RMSE value is :',np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))